In [1]:
from os.path import join
import numpy as np
from scipy import optimize
from microstruktur.signal_models import three_dimensional_models
from microstruktur.signal_models import gradient_conversions

In [2]:
bvals = np.loadtxt(join(three_dimensional_models.GRADIENT_TABLES_PATH, 'bvals_hcp_wu_minn.txt'))
gradient_directions = np.loadtxt(join(three_dimensional_models.GRADIENT_TABLES_PATH, 'bvecs_hcp_wu_minn.txt'))

In [3]:
stick = three_dimensional_models.I1Stick(lambda_par=2e-3, mu=[0, 0])

In [4]:
E = stick(bvals, gradient_directions)
E

array([ 1.        ,  0.9335884 ,  0.60888331,  0.00647561,  0.72635583,
        0.41434429,  0.32360809,  0.19819166,  0.09504999,  0.89685335,
        0.71998388,  0.02296437,  0.5116401 ,  0.23070807,  0.99901891,
        0.87608816,  1.        ,  0.77699837,  0.92441009,  0.00726072,
        0.10433723,  0.45190814,  0.24258347,  0.87642082,  0.11050229,
        0.03680408,  0.30671406,  0.25048305,  0.43244879,  0.80229888,
        0.86910295,  0.11561769,  1.        ,  0.19698608,  0.97595931,
        0.84296755,  0.16495121,  0.05832137,  0.52947699,  0.69191079,
        0.0049633 ,  0.95691371,  0.0384834 ,  0.60712385,  0.07825937,
        0.99847135,  0.43527201,  0.98953406,  1.        ,  0.2784147 ,
        0.70151996,  0.79582974,  0.82327515,  0.32211598,  0.02482972,
        0.29564166,  0.04263531,  0.59804174,  0.74817085,  0.23847658,
        0.09969115,  0.99720355,  0.06091185,  0.70564666,  1.        ,
        0.14599592,  0.07093692,  0.99862363,  0.99930743,  0.38

In [5]:
x0 = stick.parameters_to_parameter_vector(mu=[np.pi/4, 0], lambda_par=2e-3)
res = optimize.minimize(
    stick.objective_function, x0, (bvals, gradient_directions, E),
    bounds=stick.bounds_for_optimization, options={'disp': True}
)
res

      fun: 2.2193222037573134e-14
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.03347964e-04,   5.88763873e-09,   0.00000000e+00])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 212
      nit: 9
   status: 2
  success: False
        x: array([ 0.002     ,  0.        , -0.02770102])

In [6]:
stick.parameter_vector_to_parameters(res.x)

{'lambda_par': array([ 0.002]), 'mu': array([ 0.        , -0.02770102])}

In [8]:
stick2 = three_dimensional_models.I1Stick(lambda_par=2e-3, mu=[np.pi/4, 0])

In [9]:
two_sticks = three_dimensional_models.PartialVolumeCombinedMicrostrukturModel(
    [stick, stick2]
)

In [10]:
E = two_sticks(bvals, gradient_directions)
E

array([ 1.        ,  0.74950843,  0.50010035,  0.03140614,  0.84269448,
        0.69616749,  0.18568752,  0.22416797,  0.53066022,  0.4708271 ,
        0.84387703,  0.0433573 ,  0.26361524,  0.53885713,  0.57977344,
        0.91529176,  1.        ,  0.49860695,  0.86847212,  0.03750201,
        0.50975914,  0.35686711,  0.48598323,  0.81426688,  0.43123256,
        0.01985584,  0.61711469,  0.13892325,  0.33768917,  0.67512582,
        0.6196891 ,  0.55134936,  1.        ,  0.1834915 ,  0.78520574,
        0.78230336,  0.22313706,  0.08544924,  0.46802103,  0.43523245,
        0.17047706,  0.95578555,  0.02292256,  0.74301882,  0.11038927,
        0.99921583,  0.60550214,  0.5141476 ,  1.        ,  0.41014624,
        0.37419788,  0.88701509,  0.48666121,  0.47898541,  0.17528116,
        0.21721571,  0.45097505,  0.43636062,  0.82951449,  0.6047663 ,
        0.24863258,  0.87681119,  0.03607929,  0.41593407,  1.        ,
        0.33811304,  0.04672149,  0.60946846,  0.69372989,  0.57

In [18]:
two_sticks.parameter_defaults

{'I1Stick_1_lambda_par': 0.002,
 'I1Stick_1_mu': [0, 0],
 'I1Stick_2_lambda_par': 0.002,
 'I1Stick_2_mu': [0.7853981633974483, 0]}

In [19]:
x0 = two_sticks.parameters_to_parameter_vector(
    I1Stick_1_mu=[0, 0], I1Stick_1_lambda_par=1e-3,
    I1Stick_2_mu=[np.pi / 4, 0], I1Stick_2_lambda_par=2e-3,
)
res = optimize.minimize(
    two_sticks.objective_function, x0, (bvals, gradient_directions, E),
    bounds=two_sticks.bounds_for_optimization, options={'disp': True}
)
res

      fun: 1.5062876536277681e-13
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>
      jac: array([  7.01515493e-06,  -1.32186237e-07,  -1.98444069e-09,
        -4.63045764e-06,   5.14684668e-08,   4.99721295e-17])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 105
      nit: 8
   status: 0
  success: True
        x: array([  2.00000023e-03,   7.85397595e-01,  -1.38701905e-08,
         1.99999414e-03,   3.18971135e-08,   3.52204701e-16])

In [20]:
two_sticks.parameter_vector_to_parameters(res.x)

{'I1Stick_1_lambda_par': array([ 0.00199999]),
 'I1Stick_1_mu': array([  3.18971135e-08,   3.52204701e-16]),
 'I1Stick_2_lambda_par': array([ 0.002]),
 'I1Stick_2_mu': array([  7.85397595e-01,  -1.38701905e-08])}